## TREC Contextual Suggestion

✔ This notebook contains a brief EDA on phase1_requests data.


📊 Further analysis gonna be added soon ... 

---



In [1]:
import requests
from pandas import json_normalize
import pandas as pd 
import json
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

In [32]:
# Load .json data
Phase1_data = []
with open('D:/Fairness in Recommender project/datasets/TREC contextual suggestion/Phase1_requests.json') as f:
    for line in f:
        Phase1_data.append(json.loads(line))

In [33]:
# Convert data to pndas dataframe --> to perfom EDA
phase1_df = pd.DataFrame(Phase1_data)

In [35]:
phase1_df.head()

,id,body
0,1,"{'group': 'Family', 'season': 'Summer', 'trip_..."
1,2,"{'group': 'Family', 'season': 'Summer', 'trip_..."
2,4,"{'group': 'Family', 'season': 'Summer', 'trip_..."
3,5,"{'group': 'Family', 'season': 'Summer', 'trip_..."
4,8,"{'group': 'Alone', 'season': 'Autumn', 'trip_t..."


In [39]:
# convert 'body' column which a dictionary into multiple colmuns
phase1_df = pd.json_normalize(phase1_df['body'])

In [5]:
phase1_df.shape

(495, 13)

In [6]:
phase1_df.head(3)

,group,season,trip_type,duration,location.id,location.name,location.state,location.lat,location.lng,person.gender,person.age,person.id,person.preferences
0,Family,Summer,Holiday,Weekend trip,152,Chicago,IL,41.85003,-87.65005,Male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
1,Family,Summer,Holiday,Longer,210,Cape Coral,FL,26.56285,-81.94953,Male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
2,Family,Summer,Holiday,Weekend trip,151,New York City,NY,40.71427,-74.00597,Male,66,A10QUVJBDFSJYH,"[{'documentId': 'TRECCS-00354073-161', 'rating..."


In [7]:
phase1_df.describe()

,location.id,location.lat,location.lng
count,495.000000,495.000000,495.000000
mean,247.377778,36.929634,-94.343827
std,82.689414,6.542180,17.676590
min,151.000000,20.894720,-157.858330
25%,173.500000,32.500700,-100.783740
50%,223.000000,36.174970,-89.630120
75%,319.000000,40.759585,-81.675530
max,418.000000,61.218060,-68.777810


In [8]:
phase1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   group               495 non-null    object 
 1   season              495 non-null    object 
 2   trip_type           495 non-null    object 
 3   duration            495 non-null    object 
 4   location.id         495 non-null    int64  
 5   location.name       495 non-null    object 
 6   location.state      495 non-null    object 
 7   location.lat        495 non-null    float64
 8   location.lng        495 non-null    float64
 9   person.gender       495 non-null    object 
 10  person.age          495 non-null    object 
 11  person.id           495 non-null    object 
 12  person.preferences  495 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 50.4+ KB


### Data cleaning 
####  1.person.age outliers
#### 2.person.gender 
#### 3. person.age

In [9]:
phase1_df['person.age'].value_counts()

28      55
24      50
30      28
25      26
35      23
33      22
34      21
29      21
23      19
26      18
31      18
27      15
37      15
22      11
43      11
39      10
42       9
40       8
32       8
46       7
36       6
21       6
41       6
59       6
52       6
56       5
51       4
67       4
20       4
44       4
50       4
48       4
30       3
19       3
18       3
47       3
57       3
50       3
63       3
1992     3
68       2
38       2
66       2
47       2
53       2
11       1
-3       1
45       1
54       1
55       1
62       1
10       1
Name: person.age, dtype: int64

In [12]:
px.box(phase1_df['person.age'])
## Detect Outliers --> three records in : 19992 , one point: -3

In [11]:
phase1_df = phase1_df[(phase1_df['person.age'].astype(int)>0) & (phase1_df['person.age'].astype(int)<100)]
# plot gain ! 

#### 2.Person.gender

---
In raw dataset, there re two form of words "female" & "male" :

*   Lifemale, Female --> transform all of these items to : female
*   male, Male --> transform all of these items to : male

In [60]:
# check out all the possible values for gender 
## ['Male', 'Female', 'male', 'female'] --> ['male', 'female']
phase1_df['person.gender'].unique()

array(['male', 'female'], dtype=object)

In [59]:
# correct above output and run the cell above again (to see the effect)
phase1_df = phase1_df.replace({'person.gender': {'Female':'female', 'Male':'male'}})

#### 3. person.age
---
In raw dataset, content of 'age' column is mentioned as 2 different data types :

*   string
*   int


--> Solution : convert all recoreds to 'int'

In [18]:
phase1_df['person.age'].values

array([23, 23, 66, 66, 32, 32, 34, 34, 30, 30, 67, 67, 33, 29, 29, 30, 30,
       34, 22, 22, 46, 46, 35, 35, 25, 25, 37, 37, 27, 27, 24, 24, 26, 26,
       24, 24, 35, 35, 25, 25, 32, 40, 40, 50, 50, 54, 27, 27, 33, 33, 24,
       24, 28, 28, 27, 27, 33, 33, 23, 23, 33, 33, 57, 57, 25, 24, 24, 24,
       35, 35, 51, 51, 56, 56, 24, 31, 45, 46, 46, 46, 27, 28, 28, 28, 11,
       42, 43, 43, 29, 29, 31, 24, 24, 24, 28, 39, 39, 63, 31, 31, 28, 28,
       40, 22, 22, 35, 35, 30, 33, 33, 31, 25, 25, 63, 62, 24, 23, 40, 40,
       33, 27, 25, 19, 19, 24, 24, 40, 40, 24, 24, 23, 23, 43, 43, 20, 20,
       23, 29, 29, 24, 24, 25, 25, 22, 34, 33, 26, 26, 29, 29, 25, 25, 31,
       31, 42, 42, 43, 43, 28, 28, 31, 31, 68, 68, 27, 37, 37, 28, 28, 52,
       52, 24, 24, 38, 38, 29, 42, 42, 30, 30, 32, 56, 19, 47, 47, 24, 24,
       23, 23, 29, 29, 41, 23, 23, 27, 27, 31, 31, 48, 48, 24, 24, 25, 25,
       20, 27, 27, 34, 34, 28, 29, 29, 44, 44, 35, 35, 24, 24, 30, 24, 24,
       34, 34, 28, 28, 30

In [17]:
phase1_df['person.age'] = phase1_df['person.age'].astype('int')
# run abive cell again (to check all data points type-->int)

### Some visualizations

In [19]:
phase1_df['location.name'].value_counts()

New York City    25
Chicago          17
Las Vegas        15
Austin           13
San Diego        12
                 ..
Victoria          1
Farmington        1
Oklahoma City     1
Louisville        1
Little Rock       1
Name: location.name, Length: 159, dtype: int64

In [20]:
#phase1_df.groupby(by=['season','location.name']).mean(phase1_df['person.age'])
px.bar(phase1_df['location.name'].value_counts())

In [21]:
fig = px.box(phase1_df, x='trip_type',y='person.age', points='all')
fig.show()

In [22]:
fig = px.bar(phase1_df, x='trip_type', color='person.gender')
fig.show()

In [23]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(phase1_df, x="location.name", y="person.age", color="group", title="Distribution of passengers age in each state with groups hue")
fig.show()

In [24]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(phase1_df, x="location.name", color="group", title="", )
fig.show()

In [25]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(phase1_df, x="trip_type", color="duration", title="", )
fig.show()

In [23]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(phase1_df, x="location.name", color="group", title="", )
fig.show()

In [30]:
corr = phase1_df.corr()

fig = px.imshow(corr)
fig.show()

In [31]:
corr

,location.id,location.lat,location.lng
location.id,1.000000,0.084358,-0.025156
location.lat,0.084358,1.000000,-0.120486
location.lng,-0.025156,-0.120486,1.000000


In [32]:
phase1_df.head(2)

,group,season,trip_type,duration,location.id,location.name,location.state,location.lat,location.lng,person.gender,person.age,person.id,person.preferences
0,Family,Summer,Holiday,Weekend trip,152,Chicago,IL,41.85003,-87.65005,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
1,Family,Summer,Holiday,Longer,210,Cape Coral,FL,26.56285,-81.94953,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."


In [27]:
#import plotly.express as px
import numpy as np
df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(phase1_df, path=['person.gender', 'trip_type'],
                  color='person.age',
                  color_continuous_scale='RdBu',
                  #color_continuous_midpoint=np.average(df['person.age'])
                 )
fig.show()

In [28]:
fig = px.sunburst(phase1_df, path=['person.gender', 'season', 'trip_type'])
fig.show()

In [29]:
phase1_df.head(2)

,group,season,trip_type,duration,location.id,location.name,location.state,location.lat,location.lng,person.gender,person.age,person.id,person.preferences
0,Family,Summer,Holiday,Weekend trip,152,Chicago,IL,41.85003,-87.65005,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
1,Family,Summer,Holiday,Longer,210,Cape Coral,FL,26.56285,-81.94953,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."


In [48]:
fig = px.sunburst(phase1_df, path=['person.gender', 'season', 'trip_type'],
                  color = 'location.lng',color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(phase1_df['location.lng'])
                 )
fig.show() 
    

In [129]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

dur_labels = phase1_df['duration'].value_counts().index.values
dur_values = phase1_df['duration'].value_counts().values

season_labels = phase1_df['season'].value_counts().index.values
season_values = phase1_df['season'].value_counts().values

group_labels = phase1_df['group'].value_counts().index.values
group_values = phase1_df['group'].value_counts().values

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=2, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'},{'type':'domain'}]])
fig.add_trace(go.Pie(labels=["male", "female"], values=[2,5], name="Gender"),
              1, 1)
fig.add_trace(go.Pie(labels=dur_labels, values=dur_values, name="Duration"),
              1, 2)
fig.add_trace(go.Pie(labels=season_labels, values=season_values, name="Season"),
              2, 1)
fig.add_trace(go.Pie(labels=group_labels, values=group_values, name="Duration"),
              2, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Demographic Information Classificaton",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='Gender', x=0.19, y=0.82, font_size=14, showarrow=False),
                 dict(text='Duration', x=0.816, y=0.82, font_size=14, showarrow=False),
                 dict(text='Season', x=0.19, y=0.18, font_size=14, showarrow=False),
                 dict(text='Group', x=0.80, y=0.18, font_size=14, showarrow=False)
                ])
fig.show()

In [123]:
phase1_df['group'].value_counts()

Family     267
Friends    172
Alone       47
Other        9
Name: group, dtype: int64

### Encode categorical features

In [31]:
phase1_df.head()

,group,season,trip_type,duration,location.id,location.name,location.state,location.lat,location.lng,person.gender,person.age,person.id,person.preferences
0,Family,Summer,Holiday,Weekend trip,152,Chicago,IL,41.85003,-87.65005,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
1,Family,Summer,Holiday,Longer,210,Cape Coral,FL,26.56285,-81.94953,male,23,A00126103VB6TFM3EITH9,"[{'documentId': 'TRECCS-00247633-160', 'rating..."
2,Family,Summer,Holiday,Weekend trip,151,New York City,NY,40.71427,-74.00597,male,66,A10QUVJBDFSJYH,"[{'documentId': 'TRECCS-00354073-161', 'rating..."
3,Family,Summer,Holiday,Weekend trip,191,New Orleans,LA,29.95465,-90.07507,male,66,A10QUVJBDFSJYH,"[{'documentId': 'TRECCS-00354073-161', 'rating..."
4,Alone,Autumn,Business,Weekend trip,169,Orlando,FL,28.53834,-81.37924,male,32,A10VLJT7V96D9W,"[{'documentId': 'TRECCS-00354316-161', 'rating..."


In [53]:
#phase1_df = pd.json_normalize(phase1_df['body'])
preferences_df = pd.json_normalize(phase1_df['person.preferences'])
preferences_df.head()

AttributeError: 'list' object has no attribute 'values'

In [52]:
preferences_df.groupby('rating').apply(lambda x: x['tags'])

rating    
-1      4                                            NaN
        22                                           NaN
 1      19                                     [Cycling]
        21                                 [Bar-hopping]
 2      2                               [Seafood, Sushi]
        5                                        [Sport]
        7                                      [Culture]
        10                                 [Restaurants]
        13                          [Shopping for shoes]
        14                                           NaN
        25    [Shopping for food, Markets, Organic Food]
 3      0                                      [Museums]
        1                             [Farmer's markets]
        3                            [Beer, Restaurants]
        6                         [Seafood, Fine Dining]
        8                                  [Fine Dining]
        9                                   [Live Music]
        11          

In [55]:
phase1_df['person.preferences'][0]

[{'documentId': 'TRECCS-00247633-160', 'rating': 3, 'tags': ['Museums']},
 {'documentId': 'TRECCS-00018097-160',
  'rating': 3,
  'tags': ["Farmer's markets"]},
 {'documentId': 'TRECCS-00674898-160',
  'rating': 2,
  'tags': ['Seafood', 'Sushi']},
 {'documentId': 'TRECCS-00247531-160',
  'rating': 3,
  'tags': ['Beer', 'Restaurants']},
 {'documentId': 'TRECCS-00809368-160', 'rating': -1},
 {'documentId': 'TRECCS-00086637-160', 'rating': 2, 'tags': ['Sport']},
 {'documentId': 'TRECCS-00085880-160',
  'rating': 3,
  'tags': ['Seafood', 'Fine Dining']},
 {'documentId': 'TRECCS-00340169-160', 'rating': 2, 'tags': ['Culture']},
 {'documentId': 'TRECCS-00675239-160', 'rating': 3, 'tags': ['Fine Dining']},
 {'documentId': 'TRECCS-00247640-160', 'rating': 3, 'tags': ['Live Music']},
 {'documentId': 'TRECCS-00674928-160', 'rating': 2, 'tags': ['Restaurants']},
 {'documentId': 'TRECCS-00085961-160', 'rating': 3, 'tags': ['Fine Dining']},
 {'documentId': 'TRECCS-00086797-160',
  'rating': 3,
  't